# Bike Category Classification: CNN Comparison Study

This notebook compares **CNN architectures trained from scratch** vs **pretrained models fine-tuned** for bicycle category classification (5 classes: cargo, fold, hybrid, mtb, road).

## Experiment Design (32 Experiments)
- **2 Architectures**: MobileNetV2 (lightweight, suited for small datasets) & ResNet18 (skip connections)
- **4 Data Scenarios**: E1 (all), E2 (synthetic), E3 (real), E4 (balanced)
- **4 Augmentation Variants**: A1 (baseline), A2 (geometric), A3 (color), A4 (heavy)


In [ ]:
# =============================================================================
# DATASET DOWNLOAD & SETUP
# =============================================================================
import requests
import zipfile
import os
import shutil
import random
from glob import glob
from PIL import Image

# Dataset configuration
DATASET_URL = "https://github.com/michaail/bike-classifier-dataset/archive/refs/heads/main.zip"
DATASET_DIR = "dataset"
SPLIT_DIR = "dataset_split"

# CRITICAL: Fixed seed for reproducible, immutable test set
SPLIT_SEED = 42

def download_dataset(url=DATASET_URL, extract_dir=DATASET_DIR):
    """Downloads and extracts the dataset."""
    if os.path.exists(os.path.join(extract_dir, "bike-classifier-dataset-main")):
        print("Dataset already downloaded")
        return
    
    print("Downloading dataset...")
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    zip_path = "dataset.zip"
    with open(zip_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    
    print("Extracting dataset...")
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_dir)
    
    os.remove(zip_path)
    print("Dataset downloaded and extracted")

def create_deterministic_splits(source_base_path, target_base_path=SPLIT_DIR):
    """
    Creates train/val/test splits with FIXED test set.
    
    Split sizes per class (20 images each):
    - Train: 12 images
    - Val: 4 images  
    - Test: 4 images (FIXED - never changes between experiments)
    
    Total real test set: 4 × 5 classes = 20 images (as per requirements)
    """
    
    # CRITICAL: Always use same seed for identical test set
    rng = random.Random(SPLIT_SEED)
    
    source_real = os.path.join(source_base_path, 'real')
    source_syn = os.path.join(source_base_path, 'synthetic')
    
    # Split configuration: Train 12, Val 4, Test 4 = 20 total per class
    SPLIT_COUNTS = {'train': 12, 'val': 4, 'test': 4}
    
    # Clean previous splits
    if os.path.exists(target_base_path):
        shutil.rmtree(target_base_path)
    
    classes = [d for d in os.listdir(source_real) if os.path.isdir(os.path.join(source_real, d))]
    print(f"Classes detected: {classes}")
    
    def copy_files(file_list, subset_name, split_name, class_name):
        dest_dir = os.path.join(target_base_path, subset_name, split_name, class_name)
        os.makedirs(dest_dir, exist_ok=True)
        for file_path in file_list:
            shutil.copy2(file_path, dest_dir)
    
    for class_name in classes:
        real_imgs = glob(os.path.join(source_real, class_name, '*.*'))
        syn_imgs = glob(os.path.join(source_syn, class_name, '*.*'))
        
        # Sort first for determinism, then shuffle with fixed seed
        real_imgs.sort()
        syn_imgs.sort()
        rng.shuffle(real_imgs)
        rng.shuffle(syn_imgs)
        
        # Calculate splits
        train_end = SPLIT_COUNTS['train']
        val_end = train_end + SPLIT_COUNTS['val']
        test_end = val_end + SPLIT_COUNTS['test']
        
        # Real data splits
        if len(real_imgs) >= test_end:
            r_train = real_imgs[:train_end]
            r_val = real_imgs[train_end:val_end]
            r_test = real_imgs[val_end:test_end]
            
            copy_files(r_train, 'real', 'train', class_name)
            copy_files(r_val, 'real', 'val', class_name)
            copy_files(r_test, 'real', 'test', class_name)
        
        # Synthetic data splits
        if len(syn_imgs) >= test_end:
            s_train = syn_imgs[:train_end]
            s_val = syn_imgs[train_end:val_end]
            s_test = syn_imgs[val_end:test_end]
            
            copy_files(s_train, 'synthetic', 'train', class_name)
            copy_files(s_val, 'synthetic', 'val', class_name)
            copy_files(s_test, 'synthetic', 'test', class_name)
        
        # Full (combined) splits
        if len(real_imgs) >= test_end or len(syn_imgs) >= test_end:
            copy_files(r_train + s_train, 'full', 'train', class_name)
            copy_files(r_val + s_val, 'full', 'val', class_name)
            # Test set: USE ONLY REAL for consistent evaluation
            copy_files(r_test, 'full', 'test', class_name)
    
    print(f"Splits created at: {target_base_path}")
    return target_base_path

# Download and create splits
download_dataset()
source_path = os.path.join(DATASET_DIR, "bike-classifier-dataset-main")
create_deterministic_splits(source_path)

# Verify splits
print("\n📊 Dataset Split Summary:")
for subset in ['real', 'synthetic', 'full']:
    for split in ['train', 'val', 'test']:
        path = os.path.join(SPLIT_DIR, subset, split)
        if os.path.exists(path):
            total = sum(len(os.listdir(os.path.join(path, c))) for c in os.listdir(path))
            print(f"  {subset}/{split}: {total} images")

In [ ]:
# =============================================================================
# IMPORTS & CONFIGURATION
# =============================================================================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.datasets.folder as folder_module

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support, classification_report
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy
import random
import time
import os
import warnings
warnings.filterwarnings('ignore')

# Check device
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Using device: {DEVICE}")
if DEVICE.type == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")


In [ ]:
# =============================================================================
# REPRODUCIBILITY SEEDS
# =============================================================================
SEED = 42

def set_seed(seed=SEED):
    """Set all seeds for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)
print(f"Random seed set to {SEED} for reproducibility")

## Model Architectures

### Why these models?
For a **small dataset (~50-100 training images)**, large models like VGG16 (138M params) will severely overfit. Instead, we use:

1. **MobileNetV2** (~3.4M params): Designed for efficiency, uses inverted residuals and linear bottlenecks. Excellent for small datasets with pretrained weights.

2. **ResNet18** (~11M params): Uses skip connections to enable deeper training. Good balance of capacity and regularization.

Both architectures are compared:
- **From Scratch**: Random initialization, trained only on our data
- **Fine-tuned**: Pretrained on ImageNet, adapted to our task

In [ ]:
# =============================================================================
# MODEL DEFINITIONS
# =============================================================================

class ModelWrapper(nn.Module):
    """
    Unified wrapper for CNN models that returns (output, features).
    Supports both scratch and pretrained initialization.
    """
    def __init__(self, model, num_classes, feature_dim):
        super().__init__()
        self.model = model
        self.num_classes = num_classes
        self.feature_dim = feature_dim
    
    def forward(self, x):
        # For models where we need to extract features
        if hasattr(self.model, 'features'):
            # MobileNetV2 style
            features = self.model.features(x)
            features = nn.functional.adaptive_avg_pool2d(features, 1)
            features = features.view(features.size(0), -1)
            out = self.model.classifier(features)
        else:
            # ResNet style - extract features before fc
            x = self.model.conv1(x)
            x = self.model.bn1(x)
            x = self.model.relu(x)
            x = self.model.maxpool(x)
            x = self.model.layer1(x)
            x = self.model.layer2(x)
            x = self.model.layer3(x)
            x = self.model.layer4(x)
            features = self.model.avgpool(x)
            features = features.view(features.size(0), -1)
            out = self.model.fc(features)
        return out, features


def create_mobilenetv2(num_classes, pretrained=False):
    """
    Creates MobileNetV2 model.
    - ~3.4M parameters (vs VGG16's 138M)
    - Uses inverted residuals and linear bottlenecks
    - Ideal for small datasets
    """
    if pretrained:
        model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
    else:
        model = models.mobilenet_v2(weights=None)
    
    # Replace classifier for our number of classes
    in_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(in_features, num_classes)
    )
    
    return ModelWrapper(model, num_classes, in_features)


def create_resnet18(num_classes, pretrained=False):
    """
    Creates ResNet18 model.
    - ~11M parameters
    - Skip connections help with gradient flow
    - Good for small-medium datasets
    """
    if pretrained:
        model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    else:
        model = models.resnet18(weights=None)
    
    # Replace final fc layer
    in_features = model.fc.in_features
    model.fc = nn.Linear(in_features, num_classes)
    
    return ModelWrapper(model, num_classes, in_features)


# =============================================================================
# ARCHITECTURE REGISTRY
# =============================================================================

NUM_CLASSES = 5

ARCHITECTURES = {
    # From scratch (random initialization)
    'MobileNetV2_scratch': lambda: create_mobilenetv2(NUM_CLASSES, pretrained=False),
    'ResNet18_scratch': lambda: create_resnet18(NUM_CLASSES, pretrained=False),
    # Pretrained (ImageNet weights, fine-tuned)
    'MobileNetV2_pretrained': lambda: create_mobilenetv2(NUM_CLASSES, pretrained=True),
    'ResNet18_pretrained': lambda: create_resnet18(NUM_CLASSES, pretrained=True),
}

# Test model creation
print("Model Architecture Summary:")
for name, create_fn in ARCHITECTURES.items():
    model = create_fn()
    params = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"  {name}: {params:,} params ({trainable:,} trainable)")

## Data Augmentation Variants (A1-A4)

Four augmentation strategies to compare:
- **A1 (Baseline)**: Only resize + normalize (no augmentation)
- **A2 (Geometric)**: Spatial transforms (crop, flip, rotation)
- **A3 (Color)**: Color/intensity transforms (jitter, grayscale)
- **A4 (Heavy)**: Combination of all augmentations

In [ ]:
# =============================================================================
# AUGMENTATION VARIANTS (A1-A4)
# =============================================================================

# ImageNet normalization (required for pretrained models)
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

# A1: Baseline - No augmentation (only resize + normalize)
aug_A1_baseline = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# A2: Geometric - Spatial transformations
aug_A2_geometric = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# A3: Color - Color/intensity transformations  
aug_A3_color = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# A4: Heavy - All augmentations combined
aug_A4_heavy = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# Validation/Test transform (always deterministic - NO augmentation)
val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
])

# Dictionary for easy access
AUGMENTATION_VARIANTS = {
    'A1_baseline': aug_A1_baseline,
    'A2_geometric': aug_A2_geometric,
    'A3_color': aug_A3_color,
    'A4_heavy': aug_A4_heavy,
}

print("Augmentation variants defined:")
for name in AUGMENTATION_VARIANTS:
    print(f"   - {name}")

## Data Scenarios (E1-E4)

Four training data scenarios as per requirements:
- **E1 (Full)**: All available data (real + synthetic)
- **E2 (Synthetic)**: Only synthetic/generated images
- **E3 (Real)**: Only real photographs
- **E4 (Balanced)**: Real data balanced to smallest class size

In [ ]:
# =============================================================================
# DATA SCENARIOS (E1-E4)
# =============================================================================

DATA_SCENARIOS = {
    'E1_full': 'dataset_split/full',           # Real + Synthetic combined
    'E2_synthetic': 'dataset_split/synthetic', # Only synthetic
    'E3_real': 'dataset_split/real',           # Only real
    'E4_balanced': 'dataset_split/real',       # Balanced real (special handling)
}

# FIXED test set path - always use real test data for consistent evaluation
FIXED_TEST_PATH = 'dataset_split/real/test'


def get_balanced_indices(dataset, seed=SEED):
    """
    Returns indices for a balanced subset where each class has equal samples.
    Uses the minimum class count as the target.
    """
    rng = random.Random(seed)
    
    # Group indices by class
    class_indices = {}
    for idx, (_, label) in enumerate(dataset.samples):
        if label not in class_indices:
            class_indices[label] = []
        class_indices[label].append(idx)
    
    # Find minimum class size
    min_count = min(len(indices) for indices in class_indices.values())
    
    # Sample equally from each class
    balanced_indices = []
    for indices in class_indices.values():
        sampled = rng.sample(indices, min_count) if len(indices) > min_count else indices
        balanced_indices.extend(sampled)
    
    return balanced_indices, min_count


def create_data_loaders(scenario, batch_size, aug_variant='A1_baseline'):
    """
    Creates train, val, test loaders for a given scenario and augmentation.
    
    CRITICAL: Test loader ALWAYS uses the same fixed test set (real images only)
    with deterministic transforms for consistent evaluation across experiments.
    """
    train_transform = AUGMENTATION_VARIANTS.get(aug_variant, aug_A1_baseline)
    
    if scenario == 'E4_balanced':
        # Special handling for balanced dataset
        data_dir = DATA_SCENARIOS['E3_real']
        train_path = os.path.join(data_dir, 'train')
        
        full_dataset = datasets.ImageFolder(train_path, transform=train_transform)
        balanced_indices, samples_per_class = get_balanced_indices(full_dataset)
        
        train_dataset = Subset(full_dataset, balanced_indices)
        train_loader = DataLoader(train_dataset, batch_size, shuffle=True, 
                                  num_workers=2, pin_memory=True)
        train_size = len(train_dataset)
        class_names = full_dataset.classes
        
    else:
        data_dir = DATA_SCENARIOS[scenario]
        train_path = os.path.join(data_dir, 'train')
        
        train_dataset = datasets.ImageFolder(train_path, transform=train_transform)
        train_loader = DataLoader(train_dataset, batch_size, shuffle=True,
                                  num_workers=2, pin_memory=True)
        train_size = len(train_dataset)
        class_names = train_dataset.classes
    
    # Validation set - use real data with deterministic transforms
    val_path = os.path.join(DATA_SCENARIOS['E3_real'], 'val')
    val_dataset = datasets.ImageFolder(val_path, transform=val_test_transform)
    val_loader = DataLoader(val_dataset, batch_size, shuffle=False,
                           num_workers=2, pin_memory=True)
    
    # TEST SET - CRITICAL: Always the same fixed test set!
    test_dataset = datasets.ImageFolder(FIXED_TEST_PATH, transform=val_test_transform)
    test_loader = DataLoader(test_dataset, batch_size, shuffle=False,
                            num_workers=2, pin_memory=True)
    
    sizes = {
        'train': train_size,
        'val': len(val_dataset),
        'test': len(test_dataset)
    }
    
    return train_loader, val_loader, test_loader, class_names, sizes


# Verify data scenarios and fixed test set
print("Data Scenarios Summary:")
print(f"   Fixed Test Set: {FIXED_TEST_PATH}")

for scenario, path in DATA_SCENARIOS.items():
    train_path = os.path.join(path, 'train')
    if os.path.exists(train_path):
        dataset = datasets.ImageFolder(train_path, transform=val_test_transform)
        print(f"   {scenario}: {len(dataset)} train images, classes={dataset.classes}")

# Verify test set
test_dataset = datasets.ImageFolder(FIXED_TEST_PATH, transform=val_test_transform)
print(f"\nFIXED Test Set: {len(test_dataset)} images (must be 20)")
for class_name in test_dataset.classes:
    class_count = len([s for s in test_dataset.samples if test_dataset.classes[s[1]] == class_name])
    # This counts correctly
test_counts = {c: 0 for c in test_dataset.classes}
for _, label in test_dataset.samples:
    test_counts[test_dataset.classes[label]] += 1
print(f"   Per class: {test_counts}")

## Training & Evaluation Functions

Training configuration optimized for small datasets:
- **Learning rate**: 1e-3 (scratch) or 1e-4 (fine-tuning)  
- **Weight decay**: 1e-4 for regularization
- **Early stopping**: Based on validation accuracy
- **Epochs**: 30 (enough for small dataset convergence)

In [ ]:
# =============================================================================
# EXPERIMENT CONFIGURATION
# =============================================================================

EXPERIMENT_CONFIG = {
    'num_classes': 5,
    'num_epochs': 30,
    'batch_size': 16,  # Small batch for small dataset
    'learning_rate_scratch': 1e-3,    # Higher LR for training from scratch
    'learning_rate_finetune': 1e-4,   # Lower LR for fine-tuning pretrained
    'weight_decay': 1e-4,
    'patience': 7,  # Early stopping patience
}

print("Experiment Configuration:")
for k, v in EXPERIMENT_CONFIG.items():
    print(f"   {k}: {v}")

In [ ]:
# =============================================================================
# TRAINING FUNCTION
# =============================================================================

def train_model(model, train_loader, val_loader, criterion, optimizer, 
                num_epochs, device=DEVICE, patience=7, verbose=True):
    """
    Trains a model with early stopping based on validation accuracy.
    Returns the best model state and training history.
    """
    model = model.to(device)
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    
    best_val_acc = 0.0
    best_model_state = None
    epochs_without_improvement = 0
    
    for epoch in range(num_epochs):
        # ===== Training Phase =====
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs, _ = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * labels.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_loss = running_loss / max(total, 1)
        train_acc = 100.0 * correct / max(total, 1)
        
        # ===== Validation Phase =====
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs, _ = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * labels.size(0)
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        val_loss = val_loss / max(val_total, 1)
        val_acc = 100.0 * val_correct / max(val_total, 1)
        
        # Record history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        # Check for improvement
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = copy.deepcopy(model.state_dict())
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
        
        if verbose:
            print(f"Epoch [{epoch+1:2d}/{num_epochs}] "
                  f"Train: {train_loss:.4f}/{train_acc:.1f}% | "
                  f"Val: {val_loss:.4f}/{val_acc:.1f}% | "
                  f"Best: {best_val_acc:.1f}%")
        
        # Early stopping
        if epochs_without_improvement >= patience:
            if verbose:
                print(f"Early stopping at epoch {epoch+1}")
            break
    
    # Restore best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return model, history, best_val_acc

In [ ]:
# =============================================================================
# EVALUATION FUNCTION
# =============================================================================

def evaluate_model(model, test_loader, class_names, device=DEVICE):
    """
    Evaluates model on the FIXED test set.
    Returns accuracy, confusion matrix, precision, recall, F1.
    """
    model = model.to(device)
    model.eval()
    
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs, _ = model(images)
            _, predicted = outputs.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    # Metrics
    accuracy = 100.0 * (all_preds == all_labels).sum() / len(all_labels)
    cm = confusion_matrix(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='macro', zero_division=0
    )
    
    return {
        'accuracy': accuracy,
        'precision': precision * 100,
        'recall': recall * 100,
        'f1_score': f1 * 100,
        'confusion_matrix': cm,
        'predictions': all_preds,
        'labels': all_labels,
    }


def plot_confusion_matrix(cm, class_names, title='Confusion Matrix', ax=None):
    """Plots a confusion matrix."""
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(ax=ax, cmap='Blues', values_format='d')
    ax.set_title(title)
    return ax


def plot_training_history(history, title='Training History'):
    """Plots training and validation loss/accuracy curves."""
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Loss
    axes[0].plot(history['train_loss'], label='Train', linewidth=2)
    axes[0].plot(history['val_loss'], label='Val', linewidth=2)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title(f'{title} - Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Accuracy
    axes[1].plot(history['train_acc'], label='Train', linewidth=2)
    axes[1].plot(history['val_acc'], label='Val', linewidth=2)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].set_title(f'{title} - Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

print("Training & Evaluation functions defined")

## Experiment Runner

The experiment runner executes all 32 experiments systematically:
- 2 architectures × 4 data scenarios × 4 augmentation variants = 32 experiments
- Plus comparison with pretrained models adds another dimension

Each experiment:
1. Creates fresh model instance
2. Loads appropriate data loaders
3. Trains with early stopping
4. Evaluates on FIXED test set
5. Records all metrics

In [ ]:
# =============================================================================
# EXPERIMENT RUNNER
# =============================================================================

def run_single_experiment(arch_name, scenario, aug_variant, config=EXPERIMENT_CONFIG, verbose=True):
    """
    Runs a single experiment and returns results.
    """
    set_seed(SEED)  # Reset seed for reproducibility
    
    # Determine if this is a pretrained model
    is_pretrained = 'pretrained' in arch_name
    learning_rate = config['learning_rate_finetune'] if is_pretrained else config['learning_rate_scratch']
    
    print(f"\n{'='*70}")
    print(f"   Experiment: {arch_name} | {scenario} | {aug_variant}")
    print(f"   Device: {DEVICE} | LR: {learning_rate} | Pretrained: {is_pretrained}")
    print(f"{'='*70}")
    
    # Create data loaders
    train_loader, val_loader, test_loader, class_names, sizes = create_data_loaders(
        scenario, config['batch_size'], aug_variant
    )
    print(f"   Data: train={sizes['train']}, val={sizes['val']}, test={sizes['test']}")
    
    # Create model
    model = ARCHITECTURES[arch_name]()
    
    # Setup training
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=config['weight_decay'])
    
    # Train
    start_time = time.time()
    model, history, best_val_acc = train_model(
        model, train_loader, val_loader, criterion, optimizer,
        config['num_epochs'], DEVICE, config['patience'], verbose=verbose
    )
    train_time = time.time() - start_time
    
    # Evaluate on FIXED test set
    metrics = evaluate_model(model, test_loader, class_names, DEVICE)
    
    print(f"\nTest Results (on FIXED test set of {sizes['test']} images):")
    print(f"   Accuracy:  {metrics['accuracy']:.2f}%")
    print(f"   Precision: {metrics['precision']:.2f}%")
    print(f"   Recall:    {metrics['recall']:.2f}%")
    print(f"   F1 Score:  {metrics['f1_score']:.2f}%")
    print(f"   Time:      {train_time:.1f}s")
    
    return {
        'architecture': arch_name,
        'scenario': scenario,
        'augmentation': aug_variant,
        'pretrained': is_pretrained,
        'train_size': sizes['train'],
        'val_size': sizes['val'],
        'test_size': sizes['test'],
        'best_val_acc': best_val_acc,
        'test_accuracy': metrics['accuracy'],
        'test_precision': metrics['precision'],
        'test_recall': metrics['recall'],
        'test_f1': metrics['f1_score'],
        'train_time': train_time,
        'model': model,
        'history': history,
        'confusion_matrix': metrics['confusion_matrix'],
        'class_names': class_names,
    }


def run_all_experiments(architectures=None, scenarios=None, augmentations=None, 
                       config=EXPERIMENT_CONFIG, save_models=True):
    """
    Runs all experiments and returns results DataFrame.
    
    Default: 4 architectures × 4 scenarios × 4 augmentations = 64 experiments
    For just scratch comparison: 2 architectures × 4 scenarios × 4 augmentations = 32 experiments
    """
    if architectures is None:
        architectures = list(ARCHITECTURES.keys())
    if scenarios is None:
        scenarios = list(DATA_SCENARIOS.keys())
    if augmentations is None:
        augmentations = list(AUGMENTATION_VARIANTS.keys())
    
    os.makedirs('models', exist_ok=True)
    
    total = len(architectures) * len(scenarios) * len(augmentations)
    print(f"\n Running {total} experiments...")
    print(f"   Architectures: {architectures}")
    print(f"   Scenarios: {scenarios}")
    print(f"   Augmentations: {augmentations}")
    
    all_results = []
    
    for arch in architectures:
        for scenario in scenarios:
            for aug in augmentations:
                try:
                    result = run_single_experiment(arch, scenario, aug, config, verbose=False)
                    all_results.append(result)
                    
                    if save_models:
                        model_name = f"{arch}_{scenario}_{aug}.pth"
                        torch.save(result['model'].state_dict(), f"models/{model_name}")
                        
                except Exception as e:
                    print(f" ERROR in {arch}/{scenario}/{aug}: {e}")
                    all_results.append({
                        'architecture': arch,
                        'scenario': scenario,
                        'augmentation': aug,
                        'error': str(e),
                    })
    
    # Create summary DataFrame
    summary_data = []
    for r in all_results:
        if 'error' not in r:
            summary_data.append({
                'Architecture': r['architecture'],
                'Pretrained': r['pretrained'],
                'Scenario': r['scenario'],
                'Augmentation': r['augmentation'],
                'Train Size': r['train_size'],
                'Best Val Acc (%)': round(r['best_val_acc'], 2),
                'Test Acc (%)': round(r['test_accuracy'], 2),
                'Test Precision (%)': round(r['test_precision'], 2),
                'Test Recall (%)': round(r['test_recall'], 2),
                'Test F1 (%)': round(r['test_f1'], 2),
                'Train Time (s)': round(r['train_time'], 1),
            })
    
    results_df = pd.DataFrame(summary_data)
    return results_df, all_results

print(" Experiment runner configured")
print(f"   Total architectures: {len(ARCHITECTURES)}")
print(f"   Total scenarios: {len(DATA_SCENARIOS)}")
print(f"   Total augmentations: {len(AUGMENTATION_VARIANTS)}")
print(f"   Max experiments: {len(ARCHITECTURES) * len(DATA_SCENARIOS) * len(AUGMENTATION_VARIANTS)}")

## Quick Test: Single Experiment

Before running all experiments, verify everything works with a single test run.

In [ ]:
# Quick test with a single experiment (MobileNetV2 scratch, real data, baseline augmentation)
test_config = {**EXPERIMENT_CONFIG, 'num_epochs': 5}  # Short run for testing

test_result = run_single_experiment(
    'MobileNetV2_scratch', 'E3_real', 'A1_baseline', 
    config=test_config, verbose=True
)

# Plot results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion matrix
plot_confusion_matrix(
    test_result['confusion_matrix'], 
    test_result['class_names'],
    title='MobileNetV2 (scratch) - Quick Test',
    ax=axes[0]
)

# Training history
axes[1].plot(test_result['history']['train_acc'], label='Train Acc', linewidth=2)
axes[1].plot(test_result['history']['val_acc'], label='Val Acc', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Training History')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nQuick test complete! Test accuracy: {test_result['test_accuracy']:.2f}%")

## Run All 32 Experiments (Scratch Models Only)

This runs the required 32 experiments:
- 2 architectures (MobileNetV2, ResNet18) - both from scratch
- 4 data scenarios (E1-E4)
- 4 augmentation variants (A1-A4)

⏱️ Estimated time: ~15-30 minutes on GPU, ~60-120 minutes on CPU

In [ ]:
# =============================================================================
# RUN ALL 32 EXPERIMENTS (Scratch Models Only - as per requirements)
# =============================================================================

# Run experiments with scratch models only (2 arch × 4 scenarios × 4 aug = 32)
scratch_architectures = ['MobileNetV2_scratch', 'ResNet18_scratch']

results_scratch_df, results_scratch = run_all_experiments(
    architectures=scratch_architectures,
    scenarios=list(DATA_SCENARIOS.keys()),
    augmentations=list(AUGMENTATION_VARIANTS.keys()),
    config=EXPERIMENT_CONFIG,
    save_models=True
)

# Save results
results_scratch_df.to_csv('experiment_results_scratch.csv', index=False)

print("\n" + "="*80)
print("SCRATCH MODELS - RESULTS SUMMARY (32 Experiments)")
print("="*80)
print(results_scratch_df.to_string())

## Run Pretrained/Fine-tuned Comparison

Now run the same experiments with pretrained (ImageNet) models to compare transfer learning vs training from scratch.

This is crucial for small datasets - pretrained features often dramatically improve performance.

In [ ]:
# =============================================================================
# RUN PRETRAINED EXPERIMENTS (Transfer Learning Comparison)
# =============================================================================

# Run experiments with pretrained models (2 arch × 4 scenarios × 4 aug = 32)
pretrained_architectures = ['MobileNetV2_pretrained', 'ResNet18_pretrained']

results_pretrained_df, results_pretrained = run_all_experiments(
    architectures=pretrained_architectures,
    scenarios=list(DATA_SCENARIOS.keys()),
    augmentations=list(AUGMENTATION_VARIANTS.keys()),
    config=EXPERIMENT_CONFIG,
    save_models=True
)

# Save results
results_pretrained_df.to_csv('experiment_results_pretrained.csv', index=False)

print("\n" + "="*80)
print("PRETRAINED MODELS - RESULTS SUMMARY (32 Experiments)")
print("="*80)
print(results_pretrained_df.to_string())

## Results Visualization & Analysis

Compare performance across:
1. Scratch vs Pretrained models
2. Different architectures
3. Different data scenarios
4. Different augmentation strategies

In [ ]:
# =============================================================================
# COMPREHENSIVE RESULTS VISUALIZATION
# =============================================================================

# Combine all results
all_results_df = pd.concat([results_scratch_df, results_pretrained_df], ignore_index=True)
all_results_df.to_csv('experiment_results_all.csv', index=False)
all_results_list = results_scratch + results_pretrained

print(f"Total experiments completed: {len(all_results_df)}")


def plot_comprehensive_comparison(df):
    """Creates comprehensive comparison charts."""
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    
    # 1. Scratch vs Pretrained comparison
    pretrained_comparison = df.groupby('Pretrained')['Test Acc (%)'].mean()
    colors = ['#ff6b6b', '#4ecdc4']
    bars = axes[0, 0].bar(['Scratch', 'Pretrained'], pretrained_comparison.values, color=colors)
    axes[0, 0].set_title('Scratch vs Pretrained\n(Average Test Accuracy)', fontsize=12, fontweight='bold')
    axes[0, 0].set_ylabel('Accuracy (%)')
    axes[0, 0].set_ylim(0, 100)
    for bar, val in zip(bars, pretrained_comparison.values):
        axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                       f'{val:.1f}%', ha='center', fontsize=11)
    
    # 2. Architecture comparison (grouped by pretrained)
    arch_data = df.groupby(['Architecture', 'Pretrained'])['Test Acc (%)'].mean().unstack()
    arch_data.plot(kind='bar', ax=axes[0, 1], color=colors, width=0.7)
    axes[0, 1].set_title('Architecture Comparison', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylabel('Accuracy (%)')
    axes[0, 1].set_xticklabels([a.replace('_scratch', '').replace('_pretrained', '') 
                                for a in arch_data.index], rotation=0)
    axes[0, 1].legend(['Scratch', 'Pretrained'], loc='lower right')
    axes[0, 1].set_ylim(0, 100)
    
    # 3. Data Scenario comparison
    scenario_data = df.groupby(['Scenario', 'Pretrained'])['Test Acc (%)'].mean().unstack()
    scenario_data.plot(kind='bar', ax=axes[0, 2], color=colors, width=0.7)
    axes[0, 2].set_title('Data Scenario Comparison', fontsize=12, fontweight='bold')
    axes[0, 2].set_ylabel('Accuracy (%)')
    axes[0, 2].set_xticklabels(scenario_data.index, rotation=45, ha='right')
    axes[0, 2].legend(['Scratch', 'Pretrained'], loc='lower right')
    axes[0, 2].set_ylim(0, 100)
    
    # 4. Augmentation comparison
    aug_data = df.groupby(['Augmentation', 'Pretrained'])['Test Acc (%)'].mean().unstack()
    aug_data.plot(kind='bar', ax=axes[1, 0], color=colors, width=0.7)
    axes[1, 0].set_title('Augmentation Comparison', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylabel('Accuracy (%)')
    axes[1, 0].set_xticklabels(aug_data.index, rotation=45, ha='right')
    axes[1, 0].legend(['Scratch', 'Pretrained'], loc='lower right')
    axes[1, 0].set_ylim(0, 100)
    
    # 5. Heatmap: Architecture x Scenario (Scratch only)
    scratch_df = df[df['Pretrained'] == False]
    if len(scratch_df) > 0:
        # Extract base architecture name
        scratch_df = scratch_df.copy()
        scratch_df['Arch_Base'] = scratch_df['Architecture'].str.replace('_scratch', '')
        pivot = scratch_df.pivot_table(
            values='Test Acc (%)', 
            index='Arch_Base', 
            columns='Scenario', 
            aggfunc='mean'
        )
        im = axes[1, 1].imshow(pivot.values, cmap='RdYlGn', aspect='auto', vmin=0, vmax=100)
        axes[1, 1].set_xticks(range(len(pivot.columns)))
        axes[1, 1].set_xticklabels(pivot.columns, rotation=45, ha='right')
        axes[1, 1].set_yticks(range(len(pivot.index)))
        axes[1, 1].set_yticklabels(pivot.index)
        axes[1, 1].set_title('Scratch Models: Arch × Scenario', fontsize=12, fontweight='bold')
        plt.colorbar(im, ax=axes[1, 1], label='Accuracy (%)')
        for i in range(len(pivot.index)):
            for j in range(len(pivot.columns)):
                axes[1, 1].text(j, i, f'{pivot.values[i, j]:.1f}%', 
                               ha='center', va='center', fontsize=9, fontweight='bold')
    
    # 6. Heatmap: Architecture x Scenario (Pretrained)
    pretrained_df = df[df['Pretrained'] == True]
    if len(pretrained_df) > 0:
        pretrained_df = pretrained_df.copy()
        pretrained_df['Arch_Base'] = pretrained_df['Architecture'].str.replace('_pretrained', '')
        pivot = pretrained_df.pivot_table(
            values='Test Acc (%)', 
            index='Arch_Base', 
            columns='Scenario', 
            aggfunc='mean'
        )
        im = axes[1, 2].imshow(pivot.values, cmap='RdYlGn', aspect='auto', vmin=0, vmax=100)
        axes[1, 2].set_xticks(range(len(pivot.columns)))
        axes[1, 2].set_xticklabels(pivot.columns, rotation=45, ha='right')
        axes[1, 2].set_yticks(range(len(pivot.index)))
        axes[1, 2].set_yticklabels(pivot.index)
        axes[1, 2].set_title('Pretrained Models: Arch × Scenario', fontsize=12, fontweight='bold')
        plt.colorbar(im, ax=axes[1, 2], label='Accuracy (%)')
        for i in range(len(pivot.index)):
            for j in range(len(pivot.columns)):
                axes[1, 2].text(j, i, f'{pivot.values[i, j]:.1f}%', 
                               ha='center', va='center', fontsize=9, fontweight='bold')
    
    plt.tight_layout()
    return fig


def plot_all_confusion_matrices(all_results, title_prefix=''):
    """Plots confusion matrices for all experiments."""
    valid_results = [r for r in all_results if 'error' not in r]
    n = len(valid_results)
    if n == 0:
        print("No valid results to plot.")
        return
    
    n_cols = 4
    n_rows = (n + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 4*n_rows))
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    for idx, result in enumerate(valid_results):
        row, col = idx // n_cols, idx % n_cols
        ax = axes[row, col]
        
        cm = result['confusion_matrix']
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=result['class_names'])
        disp.plot(ax=ax, cmap='Blues', values_format='d', colorbar=False)
        
        pretrained_str = '(PT)' if result.get('pretrained', False) else '(S)'
        arch_short = result['architecture'].replace('_scratch', '').replace('_pretrained', '')
        ax.set_title(f"{arch_short} {pretrained_str}\n{result['scenario']}\n{result['augmentation']}", 
                    fontsize=8)
    
    # Hide empty subplots
    for idx in range(n, n_rows * n_cols):
        row, col = idx // n_cols, idx % n_cols
        axes[row, col].axis('off')
    
    plt.suptitle(f'{title_prefix} Confusion Matrices', fontsize=14, fontweight='bold')
    plt.tight_layout()
    return fig


# Generate visualizations
if len(all_results_df) > 0:
    print("\nGenerating comparison charts...")
    plot_comprehensive_comparison(all_results_df)
    plt.savefig('comparison_charts.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\nConfusion matrices for scratch models...")
    plot_all_confusion_matrices(results_scratch, 'Scratch')
    plt.savefig('confusion_matrices_scratch.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\nConfusion matrices for pretrained models...")
    plot_all_confusion_matrices(results_pretrained, 'Pretrained')
    plt.savefig('confusion_matrices_pretrained.png', dpi=150, bbox_inches='tight')
    plt.show()

## Final Summary & Best Models

In [ ]:
# =============================================================================
# FINAL SUMMARY & ANALYSIS
# =============================================================================

print("="*80)
print("FINAL EXPERIMENT SUMMARY")
print("="*80)

# Overall statistics
print(f"\nOverall Statistics:")
print(f"   Total experiments: {len(all_results_df)}")
print(f"   Test set size: 20 images (FIXED across all experiments)")

# Best results by category
print(f"\nBEST RESULTS:")

# Best scratch model
scratch_df = all_results_df[all_results_df['Pretrained'] == False]
if len(scratch_df) > 0:
    best_scratch = scratch_df.loc[scratch_df['Test Acc (%)'].idxmax()]
    print(f"\n   Best SCRATCH Model:")
    print(f"      Architecture: {best_scratch['Architecture']}")
    print(f"      Scenario: {best_scratch['Scenario']}")
    print(f"      Augmentation: {best_scratch['Augmentation']}")
    print(f"      Test Accuracy: {best_scratch['Test Acc (%)']:.2f}%")
    print(f"      F1 Score: {best_scratch['Test F1 (%)']:.2f}%")

# Best pretrained model
pretrained_df = all_results_df[all_results_df['Pretrained'] == True]
if len(pretrained_df) > 0:
    best_pretrained = pretrained_df.loc[pretrained_df['Test Acc (%)'].idxmax()]
    print(f"\n   Best PRETRAINED Model:")
    print(f"      Architecture: {best_pretrained['Architecture']}")
    print(f"      Scenario: {best_pretrained['Scenario']}")
    print(f"      Augmentation: {best_pretrained['Augmentation']}")
    print(f"      Test Accuracy: {best_pretrained['Test Acc (%)']:.2f}%")
    print(f"      F1 Score: {best_pretrained['Test F1 (%)']:.2f}%")

# Improvement from pretraining
if len(scratch_df) > 0 and len(pretrained_df) > 0:
    avg_scratch = scratch_df['Test Acc (%)'].mean()
    avg_pretrained = pretrained_df['Test Acc (%)'].mean()
    improvement = avg_pretrained - avg_scratch
    print(f"\nTransfer Learning Improvement:")
    print(f"   Average Scratch Accuracy: {avg_scratch:.2f}%")
    print(f"   Average Pretrained Accuracy: {avg_pretrained:.2f}%")
    print(f"   Improvement: +{improvement:.2f}%")

# Summary by scenario
print(f"\nAverage Accuracy by Data Scenario:")
scenario_summary = all_results_df.groupby(['Scenario', 'Pretrained'])['Test Acc (%)'].mean().unstack()
print(scenario_summary.to_string())

# Summary by augmentation
print(f"\nAverage Accuracy by Augmentation:")
aug_summary = all_results_df.groupby(['Augmentation', 'Pretrained'])['Test Acc (%)'].mean().unstack()
print(aug_summary.to_string())

# Save comprehensive results
all_results_df.to_csv('experiment_results_final.csv', index=False)
print(f"\nResults saved to experiment_results_final.csv")

# Save best model
valid_results = [r for r in all_results_list if 'error' not in r and 'test_accuracy' in r]
if valid_results:
    best = max(valid_results, key=lambda x: x['test_accuracy'])
    best_name = f"BEST_{best['architecture']}_{best['scenario']}_{best['augmentation']}.pth"
    torch.save(best['model'].state_dict(), f"models/{best_name}")
    print(f"🏆 Best model saved: models/{best_name}")
    print(f"   Test Accuracy: {best['test_accuracy']:.2f}%")

## Conclusions

### Key Findings:

1. **Pretrained vs Scratch**: For small datasets (~50-100 training images), pretrained models with fine-tuning significantly outperform models trained from scratch. Transfer learning leverages features learned from millions of ImageNet images.

2. **Architecture Choice**: MobileNetV2 and ResNet18 are appropriate for this dataset size. VGG16 (138M params) would severely overfit with so few samples.

3. **Data Scenarios**: 
   - E3 (Real only) typically performs best since test set is real images
   - E1 (Full) can help when augmented well
   - E2 (Synthetic) shows the domain gap between synthetic and real images
   - E4 (Balanced) ensures no class imbalance issues

4. **Augmentation Impact**: Heavy augmentation (A4) often helps scratch models more than pretrained models, which already have robust features.

### Recommendations:
- For production: Use **pretrained MobileNetV2/ResNet18** with heavy augmentation
- For limited compute: MobileNetV2 offers best accuracy/parameter ratio
- Always validate on real images even when training includes synthetic data